<a href="https://colab.research.google.com/github/namrqthakaipa/gcp-ai-agents/blob/main/vertexai_ai_agent_engine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Copyright Psitron

# Intro to Building and Deploying an Agent with Agent Engine in Vertex AI

## Overview

### Gemini

Gemini is a family of generative AI models developed by Google DeepMind that is designed for multimodal use cases.

### Function Calling in Gemini

[Function calling](https://cloud.google.com/vertex-ai/docs/generative-ai/multimodal/function-calling) lets developers create a description of a function in their code, then pass that description to a language model in a request. The response from the model includes the name of a function that matches the description and the arguments to call it with.

### Agent Engine in Vertex AI

[Agent Engine](https://cloud.google.com/vertex-ai/generative-ai/docs/agent-engine/overview) is a managed service that helps you to build and deploy an agent framework. It gives you the flexibility to choose how much reasoning you want to delegate to the LLM and how much you want to handle with customized code. You can define Python functions that get used as tools via [Gemini Function Calling](https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/function-calling). Agent Engine integrates closely with the Python SDK for the Gemini model in Vertex AI, and it can manage prompts, agents, and examples in a modular way. Agent Engine is compatible with LangChain, LlamaIndex, or other Python frameworks.

<img width="60%" src="https://storage.googleapis.com/github-repo/generative-ai/gemini/agent-engine/images/agent-engine-overview.png" alt="Agent Engine on Vertex AI" />

### Objectives

In this tutorial, you will learn how to build and deploy an agent (model, tools, and reasoning) using the Vertex AI SDK for Python.

You'll build and deploy an agent that uses the Gemini model, Python functions as tools, and LangChain for orchestration.

You will complete the following tasks:

- Install the Vertex AI SDK for Python
- Use the Vertex AI SDK to build components of a simple agent
- Test your agent locally before deploying
- Deploy and test your agent on Vertex AI
- Customize each layer of your agent (model, tools, orchestration)

### Costs

This tutorial uses billable components of Google Cloud:

- Vertex AI

Learn about [Vertex AI pricing](https://cloud.google.com/vertex-ai/pricing) and use the [Pricing Calculator](https://cloud.google.com/products/calculator/) to generate a cost estimate based on your projected usage.


## Getting Started


### Install Vertex AI SDK for Python

Install the latest version of the Vertex AI SDK for Python as well as extra dependencies related to Agent Engine and LangChain:

In [1]:
%pip install --upgrade --quiet \
    "google-cloud-aiplatform[agent_engines,langchain]" \
    cloudpickle==3.0.0 \
    "pydantic>=2.10" \
    requests

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.5/229.5 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.2/100.2 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 42.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.2/148.2 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.4/118.4 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.2/196.2 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.2/44.2 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.0/50.0 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.1/42.1 MB 23.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.7/4

### Restart current runtime

To use the newly installed packages in this Jupyter runtime, you must restart the runtime. You can do this by running the cell below, which will restart the current kernel.

In [2]:
# Restart kernel after installs so that your environment can access the new packages
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

<div class="alert alert-block alert-warning">
<b>⚠️ The kernel is going to restart. Please wait until it is finished before continuing to the next step. ⚠️</b>
</div>


### Authenticate your notebook environment (Colab only)

If you are running this notebook on Google Colab, run the following cell to authenticate your environment. This step is not required if you are using [Vertex AI Workbench](https://cloud.google.com/vertex-ai-workbench).

In [1]:
import sys

if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user()

### Set Google Cloud project information and initialize Vertex AI SDK

To get started using Vertex AI, you must have an existing Google Cloud project and [enable the Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com).

Learn more about [setting up a project and a development environment](https://cloud.google.com/vertex-ai/docs/start/cloud-environment).

In [2]:
PROJECT_ID = "ai-agent-vertex-461013"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}
STAGING_BUCKET = "gs://aiagentvertexai"  # @param {type:"string"}

import vertexai

vertexai.init(project=PROJECT_ID, location=LOCATION, staging_bucket=STAGING_BUCKET)

## Example: Build and deploy an agent

### Import libraries

In [3]:
from vertexai import agent_engines
from vertexai.preview.reasoning_engines import LangchainAgent

### Define model

As you construct your agent from the bottom up, the first component deals with which generative model you want to use in your agent.

<img width="40%" src="https://storage.googleapis.com/github-repo/generative-ai/gemini/agent-engine/images/agent-stack-1.png" alt="Components of an agent in Agent Engine on Vertex AI" />

Here you'll use the Gemini 2.0 model:

In [4]:
model = "gemini-2.0-flash"

### Define Python functions (tools)

The second component of your agent includes tools and functions, which enable the generative model to interact with external systems, databases, document stores, and other APIs so that the model can get the most up-to-date information or take action with those systems.

<img width="40%" src="https://storage.googleapis.com/github-repo/generative-ai/gemini/agent-engine/images/agent-stack-2.png" alt="Components of an agent in Agent Engine on Vertex AI" />

In this example, you'll define a function called `get_exchange_rate` that uses the `requests` library to retrieve real-time currency exchange information from an API:

In [5]:
def get_exchange_rate(
    currency_from: str = "USD",
    currency_to: str = "INR",
    currency_date: str = "latest",
):
    """Retrieves the exchange rate between two currencies on a specified date."""
    import requests

    response = requests.get(
        f"https://api.frankfurter.app/{currency_date}",
        params={"from": currency_from, "to": currency_to},
    )
    return response.json()

Test the function with sample inputs to ensure that it's working as expected:

In [6]:
get_exchange_rate(currency_from="USD", currency_to="INR")

{'amount': 1.0, 'base': 'USD', 'date': '2025-06-05', 'rates': {'INR': 85.83}}

### Define agent

The third component of your agent involves adding a reasoning layer, which helps your agent use the tools that you provided to help the end user achieve a higher-level goal.

<img width="40%" src="https://storage.googleapis.com/github-repo/generative-ai/gemini/agent-engine/images/agent-stack-3.png" alt="Components of an agent in Agent Engine on Vertex AI" />

If you were to use Gemini and Function Calling on their own without a reasoning layer, you would need to handle the process of calling functions and APIs in your application code, and you would need to implement retries and additional logic to ensure that your function calling code is resilient to failures and malformed requests.

Here, you'll use the LangChain agent template provided in the Vertex AI SDK for Agent Engine, which brings together the model, tools, and reasoning that you've built up so far:

In [7]:
agent = LangchainAgent(
    model=model,
    tools=[get_exchange_rate],
    agent_executor_kwargs={"return_intermediate_steps": True},
)

Now we can test the model and agent behavior to ensure that it's working as expected before we deploy it:

### Test your agent locally

With all of the core components of your agent in place, you can send a prompt to your agent using `.query` to test that it's working as expected, including the intermediate steps that the agent performed between the input prompt and the generated summary output. In the default mode, the agent processes your input and returns the **entire agent output in a single response when complete**:

In [8]:
agent.query(input="What's the exchange rate from US dollars to INR currency today?")

{'input': "What's the exchange rate from US dollars to INR currency today?",
 'output': 'The exchange rate from USD to INR is 85.83 as of today.',
 'intermediate_steps': [[{'lc': 1,
    'type': 'constructor',
    'id': ['langchain', 'schema', 'agent', 'ToolAgentAction'],
    'kwargs': {'tool': 'get_exchange_rate',
     'tool_input': {},
     'log': '\nInvoking: `get_exchange_rate` with `{}`\n\n\n',
     'type': 'AgentActionMessageLog',
     'message_log': [{'lc': 1,
       'type': 'constructor',
       'id': ['langchain', 'schema', 'messages', 'AIMessageChunk'],
       'kwargs': {'content': '',
        'additional_kwargs': {'function_call': {'name': 'get_exchange_rate',
          'arguments': '{}'}},
        'response_metadata': {'safety_ratings': [],
         'usage_metadata': {},
         'finish_reason': 'STOP',
         'model_name': 'gemini-2.0-flash'},
        'type': 'AIMessageChunk',
        'id': 'run--fa652b35-bb65-49ef-bef1-4d8fa41fcab5',
        'tool_calls': [{'name': 'get

In addition to the default query mode, the `.stream_query` method allows you to **see the agent's intermediate steps and final output from the chain**.

Instead of waiting for the agent to complete all sub-tasks, the agent sends back the response in **chunks as it's being generated**:

In [9]:
message_types = {"actions": "Action", "messages": "Message", "output": "Output"}
for chunk in agent.stream_query(
    input="What's the exchange rate from US dollars to INR currency today?"
):
    for key, label in message_types.items():
        if key in chunk:
            print("\n------\n")
            print(f"{label}:")
            print()
            print(chunk[key])


------

Action:

[{'lc': 1, 'type': 'constructor', 'id': ['langchain', 'schema', 'agent', 'ToolAgentAction'], 'kwargs': {'tool': 'get_exchange_rate', 'tool_input': {'currency_from': 'USD', 'currency_to': 'INR'}, 'log': "\nInvoking: `get_exchange_rate` with `{'currency_from': 'USD', 'currency_to': 'INR'}`\n\n\n", 'type': 'AgentActionMessageLog', 'message_log': [{'lc': 1, 'type': 'constructor', 'id': ['langchain', 'schema', 'messages', 'AIMessageChunk'], 'kwargs': {'content': '', 'additional_kwargs': {'function_call': {'name': 'get_exchange_rate', 'arguments': '{"currency_from": "USD", "currency_to": "INR"}'}}, 'response_metadata': {'safety_ratings': [], 'usage_metadata': {}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash'}, 'type': 'AIMessageChunk', 'id': 'run--6af6b032-3882-4fa2-836f-404d9a21acbf', 'tool_calls': [{'name': 'get_exchange_rate', 'args': {'currency_from': 'USD', 'currency_to': 'INR'}, 'id': 'a88c99d3-3241-498b-b103-eaa4ea61462b', 'type': 'tool_call'}], 'usage_me

This allows you to observe the agent's actions in real-time (such as function calls, and intermediate steps), which is helpful for debugging purposes or for providing real-time updates to the end user.

### Deploy your agent on Vertex AI

Now that you've specified a model, tools, and reasoning for your agent and tested it out, you're ready to deploy your agent as a remote service in Vertex AI!

<img width="40%" src="https://storage.googleapis.com/github-repo/generative-ai/gemini/agent-engine/images/agent-stack-4.png" alt="Components of an agent in Agent Engine on Vertex AI" />

You can re-define the agent to avoid any stateful information in the agent due to our testing in the previous cell:

In [10]:
agent = LangchainAgent(
    model=model,
    tools=[get_exchange_rate],
)

Now you're ready to deploy your agent to Agent Engine in Vertex AI by calling `agent_engines.create()` along with:

1. The instance of your agent class
2. The Python packages and versions that your agent requires at runtime, similar to how you would define packages and versions in a `requirements.txt` file.

In [11]:
remote_agent = agent_engines.create(
    agent,
    requirements=[
        "google-cloud-aiplatform[agent_engines,langchain]",
        "cloudpickle==3.0.0",
        "pydantic>=2.10",
        "requests",
    ],
)

INFO:vertexai.agent_engines:Identified the following requirements: {'pydantic': '2.11.5', 'google-cloud-aiplatform': '1.96.0', 'cloudpickle': '3.0.0'}
INFO:vertexai.agent_engines:The final list of requirements: ['google-cloud-aiplatform[agent_engines,langchain]', 'cloudpickle==3.0.0', 'pydantic>=2.10', 'requests']
INFO:vertexai.agent_engines:Creating bucket aiagentvertexai in location='us-central1'
INFO:vertexai.agent_engines:Wrote to gs://aiagentvertexai/agent_engine/agent_engine.pkl
INFO:vertexai.agent_engines:Writing to gs://aiagentvertexai/agent_engine/requirements.txt
INFO:vertexai.agent_engines:Creating in-memory tarfile of extra_packages
INFO:vertexai.agent_engines:Writing to gs://aiagentvertexai/agent_engine/dependencies.tar.gz
INFO:vertexai.agent_engines:Creating AgentEngine
INFO:vertexai.agent_engines:Create AgentEngine backing LRO: projects/409401031034/locations/us-central1/reasoningEngines/5316965553041047552/operations/5896675740161671168
INFO:vertexai.agent_engines:View 

Now you can send a prompt to your remote agent using `.query` to test that it's working as expected:

In [12]:
remote_agent.query(
    input="What's the exchange rate from US dollars to Yen currency today?"
)

{'input': "What's the exchange rate from US dollars to Yen currency today?",
 'output': "Today's exchange rate is 1 US dollar equals 143.27 Japanese Yen.\n"}

Or you can stream the results back from the remote agent interactively using `.stream_query`:

In [13]:
message_types = {"actions": "Action", "messages": "Message", "output": "Output"}
for chunk in remote_agent.stream_query(
    input="What's the exchange rate from US dollars to SEK currency today?"
):
    for key, label in message_types.items():
        if key in chunk:
            print("\n------\n")
            print(f"{label}:")
            print()
            print(chunk[key])


------

Action:

[{'lc': 1, 'type': 'constructor', 'id': ['langchain', 'schema', 'agent', 'ToolAgentAction'], 'kwargs': {'tool': 'get_exchange_rate', 'tool_input': {'currency_to': 'SEK'}, 'log': "\nInvoking: `get_exchange_rate` with `{'currency_to': 'SEK'}`\n\n\n", 'type': 'AgentActionMessageLog', 'message_log': [{'lc': 1, 'type': 'constructor', 'id': ['langchain', 'schema', 'messages', 'AIMessageChunk'], 'kwargs': {'content': '', 'additional_kwargs': {'function_call': {'name': 'get_exchange_rate', 'arguments': '{"currency_to": "SEK"}'}}, 'response_metadata': {'safety_ratings': [], 'usage_metadata': {}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash'}, 'type': 'AIMessageChunk', 'id': 'run--1b088f80-f081-4f87-8928-53c0c3aefeef', 'tool_calls': [{'name': 'get_exchange_rate', 'args': {'currency_to': 'SEK'}, 'id': '8cf93190-bcc6-48d4-a0ac-1965b07e4260', 'type': 'tool_call'}], 'usage_metadata': {'input_tokens': 48, 'output_tokens': 9, 'total_tokens': 57}, 'tool_call_chunks': [{'na

### Querying your deployed agent

You've now deployed your agent and can [interact with it in multiple ways](https://cloud.google.com/vertex-ai/generative-ai/docs/agent-engine/use/overview), both within this notebook and from other applications or environments. The primary methods for accessing your deployed agent are via the Python client library or through REST API calls. Here's an overview of both methods:

**Method 1: Reusing within this notebook or another Python environment**

You can directly reuse and query the `remote_agent` instance you created in this notebook.

Or, you can instantiate a new instance in another notebook or Python script. To do this, you'll need to retrieve your deployed agent's resource name that uniquely identifies your agent, which is a string that includes the project, location, and Agent Engine ID. You can retrieve it by running the following code in the notebook or environment where you created your agent:

In [14]:
remote_agent.resource_name

'projects/409401031034/locations/us-central1/reasoningEngines/5316965553041047552'

Use the resource name to load the agent in your other notebook or Python script, then query the remote agent as usual:

In [15]:
from vertexai import agent_engines

AGENT_ENGINE_RESOURCE_NAME = "projects/409401031034/locations/us-central1/reasoningEngines/5316965553041047552"  # Replace with the resource name of your deployed Agent Engine

remote_agent = agent_engines.get(AGENT_ENGINE_RESOURCE_NAME)
response = remote_agent.query(input="What's the exchange rate from US dollars to SEK currency today?")
print(response)

{'input': "What's the exchange rate from US dollars to SEK currency today?", 'output': 'The current exchange rate is 1 USD to 9.5798 SEK.'}


**Method 2: Accessing from other environments via REST API**

Beyond the Python client library, your deployed Vertex AI agent can be [queried using REST API calls](https://cloud.google.com/vertex-ai/generative-ai/docs/agent-engine/use/overview), including:

- Python: You can use Python's `requests` library or similar tools to make HTTP calls to the Vertex AI REST API.
- cURL: A command-line tool, cURL allows you to send HTTP requests directly. This is useful for testing and debugging.
- Other Programming Languages: If you prefer a different language for your application, you can use its native HTTP client library to make REST API calls.

In summary, you have access to your deployed agent through the Python client library within Python environments, and more universally through its REST API via tools and programming languages of your choosing.

```
curl \
-H "Authorization: Bearer $(gcloud auth print-access-token)" \
-H "Content-Type: application/json" \
https://LOCATION-aiplatform.googleapis.com/v1/projects/PROJECT_ID/locations/LOCATION/reasoningEngines/kjklkl:query -d '{
  "input": {
    "class_method": "query",
    "input": {"messages": [
      ("user", "What is the exchange rate from US dollars to Swedish currency?")
    ]},
  }
}'
```

Sample
```
curl \
-H "Authorization: Bearer $(gcloud auth print-access-token)" \
-H "Content-Type: application/json" \
https://us-central1-aiplatform.googleapis.com/v1/projects/aiagentvertexai/locations/us-central1/reasoningEngines/8189839902838358016:query \
-d '{
  "input": {
    "input": "What is the exchange rate from US dollars to INR currency?"
  }
}'

```

Post Man:
## 📬 Postman Setup Instructions

---

### 1. **Method & URL**

- **Method**: `POST`
- **URL**:  
https://us-central1-aiplatform.googleapis.com/v1/projects/aiagentvertexai/locations/us-central1/reasoningEngines/8189839902838358016:query

---

### 2. **Authorization**

- **Type**: Bearer Token

#### How to Get Token:

Run the following command in your terminal:

```bash
gcloud auth print-access-token
```
In Postman:
Go to the Authorization tab

Set Type to Bearer Token

Paste the token in the Token field

### 3. Headers
Add the following headers if not using the Authorization tab:

Key	Value
Authorization	Bearer YOUR_ACCESS_TOKEN
Content-Type	application/json

### 4. Body
Go to the Body tab
Select raw
Choose JSON from the format dropdown
Paste the following JSON payload:

```json
{
  "input": {
    "input": "What is the exchange rate from US dollars to INR currency?"
  }
}
```

## Cleaning up

After you've finished, it's a good practice to clean up your cloud resources. You can delete the deployed Agent Engine instance to avoid any unexpected charges on your Google Cloud account.

In [16]:
remote_agent.delete()

INFO:vertexai.agent_engines:Delete Agent Engine backing LRO: projects/409401031034/locations/us-central1/operations/6182091366546276352
INFO:vertexai.agent_engines:Agent Engine deleted. Resource name: projects/409401031034/locations/us-central1/reasoningEngines/5316965553041047552
